# Volume 3: Discrete Hidden Markov Models
    Samuel
    Goldrup
    14 February 2023

In [1]:
import numpy as np
import copy
import string
import codecs

In [2]:
class HMM(object):
    """
    Finite state space hidden markov model.
    """
    
    # Problem 1
    def __init__(self):
        """
        Initialize an HMM with parameters A, B, and pi, initialized as 
        None objects.
        """
        self.pi = None
        self.A = None
        self.B = None    
    
    # Problem 2
    def _forward(self, z):
        """
        Compute the scaled forward probability matrix and scaling factors.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        
        Returns
        -------
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix
        
        c : ndarray of shape (T,)
            The scaling factors c = [c_0, c_1, ..., c_{T-1}]
        """
        T = z.shape[0] #use shape attribute
        n = self.pi.shape[0]
        alphas = np.empty((T,n))
        c = np.empty(T)
        t = 0
        
        alphas[0] = np.array([self.pi[i]*self.B[z[t],i] for i in range(n)])
        c[0] = 1 / np.sum(alphas[0])
        alphas[0] = np.multiply(c[0],alphas[0])
        
        
        for t in range(1,T):
            for i in range(n):
                    alphas[t,i] = np.sum([alphas[t-1,j]*self.A[i,j]*self.B[z[t],i] for j in range(n)])
            ct = 1 / np.sum(alphas[t])
            c[t] = ct
            alphas[t] = np.multiply(ct,alphas[t])
        
        c = np.array(c)
        
        return alphas, c
        
    
    # Problem 3
    def _backward(self, z, c):
        """
        Compute the scaled backward probability matrix.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix
        """
        T = z.shape[0]
        n = self.pi.shape[0]
        beta = np.empty((T,n))
        
        beta[-1] = np.ones(n)*c[-1]
        
        for t in range(T-2,-1,-1):
            beta[t] = [np.sum([self.A[i,j]*beta[t+1,i]*self.B[z[t+1],i] for i in range(n)]) for j in range(n)]
            beta[t] *= c[t]          
        
        return beta
    
    
    # Problem 4
    def _xi(self, z, alpha, beta, c):
        """
        Compute the xi and gamma probabilities.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix from the backward pass
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        T = z.shape[0]
        n = self.pi.shape[0]
        xi = np.empty((T-1,n,n))
        gamma = np.empty((T,n))
        
        #nested list comprehensions
        for t in range(T-1):
            xi[t] = np.array([[alpha[t,i] * self.A[j,i] * self.B[z[t+1],j] * beta[t+1,j] for j in range(n)] for i in range(n)])
            gamma[t] = np.array([(alpha[t,i]*beta[t,i])/c[t] for i in range(n)])
        gamma[T-1] = np.array([(alpha[T-1,i]*beta[T-1,i])/c[T-1] for i in range(n)])
        
        return xi, gamma
    
    # Problem 5
    def _estimate(self, z, xi, gamma):
        """
        Estimate better parameter values and update self.pi, self.A, and
        self.B in place.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        T = z.shape[0]
        n = self.pi.shape[0]
        
        #array broadcasting this
        self.pi = gamma[0]
        self.A = np.sum(xi,axis=0).T/np.sum(gamma[:-1,:],axis=0)
        B = np.zeros_like(self.B)
        for i in range(self.B.shape[0]):
            B[i] = np.sum([gamma[t]*int(z[t]==i) for t in range(T)],axis=0)
        
        self.B = B/np.sum(gamma,axis=0)   
        
        return self

    
    # Problem 6
    def fit(self, z, pi, A, B, max_iter=100, tol=1e-5):
        """
        Fit the model parameters to a given observation sequence.

        Parameters
        ----------
        z : ndarray of shape (T,)
            Observation sequence on which to train the model.
        pi : Stochastic ndarray of shape (n,)
            Initial state distribution
        A : Stochastic ndarray of shape (n, n)
            Initial state transition matrix
        B : Stochastic ndarray of shape (m, n)
            Initial state observation matrix
        max_iter : int
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        self.pi = pi
        self.A = A
        self.B = B
        
        T = z.shape[0]
        N = self.pi.shape[0]
        M = self.B.shape[0]
        
        alpha, c = self._forward(z)
        old_p = -np.log(c).sum()
        for i in range(max_iter):
            print(i)
            alpha, c = self._forward(z)
            beta = self._backward(z,c)
            xi,gamma = self._xi(z,alpha,beta,c)
            self._estimate(z,xi,gamma)
            alpha,c = self._forward(z)
            new_p = -np.log(c).sum()
            
            if abs(new_p - old_p) < tol:
                break
            else:
                old_p = new_p
                
        return self

In [3]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case

    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)

    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))

    # convert the data to a NumPy array of symbols
    a = np.array(list(data))

    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}

    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)

    return symbols, obs_sequence

In [4]:
###TEST CODE###
pi = np.array([.6, .4]) #unit test case
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z = np.array([0, 1, 0, 2])


h = HMM()
h.pi = pi
h.A = A
h.B = B
alpha, c = h._forward(z)
print(-np.log(c).sum())

beta = h._backward(z, c)
print(beta)

-4.642913590898749
[[3.1361635  2.89939354]
 [2.86699344 4.39229044]
 [3.898812   2.66760821]
 [3.56816483 3.56816483]]


In [5]:
xi, gamma = h._xi(z, alpha, beta, c)
print(xi)
print(gamma)

[[[0.14166321 0.0465066 ]
  [0.37776855 0.43406164]]

 [[0.17015868 0.34927307]
  [0.05871895 0.4218493 ]]

 [[0.21080834 0.01806929]
  [0.59317106 0.17795132]]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [6]:
h._estimate(z, xi, gamma)

In [7]:
h.pi,h.A,h.B

(array([0.18816981, 0.81183019]),
 array([[0.55807991, 0.49898142],
        [0.44192009, 0.50101858]]),
 array([[0.23961928, 0.70056364],
        [0.29844534, 0.21268397],
        [0.46193538, 0.08675238]]))

## Problem 7

Train a HMM on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values ($26$ lower case characters and $1$ whitespace character), and run for $150$ iterations with the deffault value for `tol`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [8]:
symbols, obs = prep_data('declaration.txt')
M, N = len(set(obs)), 2

pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T #random initializations
B = np.random.dirichlet(np.ones(M), size=N).T

hmm = HMM()
hmm.fit(obs,pi,A,B)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [9]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], hmm.B[i,0], hmm.B[i,1]))


 , 0.0000, 0.2815
a, 0.0003, 0.1037
b, 0.0286, 0.0005
c, 0.0568, 0.0000
d, 0.0779, 0.0000
e, 0.0005, 0.1868
f, 0.0556, 0.0000
g, 0.0402, 0.0000
h, 0.0016, 0.0749
i, 0.0000, 0.0978
j, 0.0049, 0.0000
k, 0.0033, 0.0007
l, 0.0544, 0.0113
m, 0.0441, 0.0003
n, 0.0708, 0.0553
o, 0.0000, 0.1118
p, 0.0344, 0.0058
q, 0.0019, 0.0000
r, 0.1313, 0.0001
s, 0.1310, 0.0118
t, 0.1977, 0.0000
u, 0.0000, 0.0455
v, 0.0229, 0.0000
w, 0.0300, 0.0000
x, 0.0028, 0.0000
y, 0.0079, 0.0121
z, 0.0012, 0.0000


We see zero/near-zero entries in the second column of B.

## Problem 8

Repeat the previous calculation with $3$ hidden states and again with $4$ hidden states. Interpret/explain your results.

In [10]:
N=3
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
hmm = HMM()
hmm.fit(obs,pi,A,B)
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}".format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 , 0.0000, 0.0000, 0.8211
a, 0.0000, 0.1132, 0.0084
b, 0.0000, 0.0232, 0.0000
c, 0.0016, 0.0380, 0.0157
d, 0.0861, 0.0143, 0.0048
e, 0.2556, 0.0752, 0.0000
f, 0.0429, 0.0182, 0.0083
g, 0.0336, 0.0134, 0.0015
h, 0.0342, 0.0672, 0.0000
i, 0.0000, 0.0850, 0.0641
j, 0.0000, 0.0038, 0.0003
k, 0.0010, 0.0026, 0.0007
l, 0.0244, 0.0425, 0.0007
m, 0.0228, 0.0191, 0.0104
n, 0.1075, 0.0609, 0.0011
o, 0.0493, 0.0993, 0.0000
p, 0.0000, 0.0264, 0.0189
q, 0.0000, 0.0012, 0.0008
r, 0.0912, 0.0492, 0.0171
s, 0.1369, 0.0374, 0.0188
t, 0.0705, 0.1181, 0.0027
u, 0.0000, 0.0508, 0.0006
v, 0.0000, 0.0165, 0.0040
w, 0.0047, 0.0212, 0.0000
x, 0.0000, 0.0022, 0.0000
y, 0.0376, 0.0000, 0.0000
z, 0.0000, 0.0010, 0.0000


In [11]:
N=4
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
hmm = HMM()
hmm.fit(obs,pi,A,B)
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}".format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2], hmm.B[i,3]))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 , 0.0100, 0.2508, 0.5459, 0.0385
a, 0.0000, 0.0272, 0.0000, 0.1801
b, 0.0291, 0.0164, 0.0000, 0.0000
c, 0.0557, 0.0331, 0.0000, 0.0000
d, 0.0995, 0.0000, 0.0017, 0.0000
e, 0.0000, 0.0000, 0.0000, 0.3511
f, 0.0620, 0.0175, 0.0006, 0.0000
g, 0.0498, 0.0000, 0.0034, 0.0000
h, 0.0000, 0.0000, 0.2275, 0.0000
i, 0.0000, 0.0261, 0.0714, 0.1244
j, 0.0000, 0.0000, 0.0104, 0.0000
k, 0.0027, 0.0000, 0.0047, 0.0000
l, 0.0456, 0.0334, 0.0450, 0.0000
m, 0.0438, 0.0209, 0.0040, 0.0000
n, 0.0268, 0.3077, 0.0028, 0.0000
o, 0.0000, 0.0055, 0.0000, 0.2066
p, 0.0449, 0.0141, 0.0042, 0.0000
q, 0.0023, 0.0000, 0.0002, 0.0000
r, 0.0853, 0.0927, 0.0568, 0.0000
s, 0.1213, 0.0983, 0.0025, 0.0157
t, 0.2440, 0.0003, 0.0179, 0.0

The results are interesting. Going from 3 to 4 states, our B matrix seems to gain more sparsity. 

## Problem 9

Repeat the same calculation with `WarAndPeace.txt` for $2$ and $3$ hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [12]:
symbols, obs = prep_data('WarAndPeace.txt')
M = len(set(obs))

N=2
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
hmm = HMM()
hmm.fit(obs,pi,A,B)

for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], hmm.B[i,0], hmm.B[i,1]))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 , 0.0879, 0.2149
а, 0.1752, 0.0000
б, 0.0000, 0.0251
в, 0.0000, 0.0657
г, 0.0000, 0.0297
д, 0.0001, 0.0386
е, 0.1433, 0.0172
ж, 0.0000, 0.0140
з, 0.0000, 0.0253
и, 0.1323, 0.0007
й, 0.0000, 0.0150
к, 0.0012, 0.0498
л, 0.0000, 0.0721
м, 0.0000, 0.0382
н, 0.0000, 0.0976
о, 0.2397, 0.0000
п, 0.0064, 0.0346
р, 0.0000, 0.0599
с, 0.0281, 0.0513
т, 0.0000, 0.0782
у, 0.0588, 0.0000
ф, 0.0003, 0.0018
х, 0.0000, 0.0111
ц, 0.0000, 0.0050
ч, 0.0038, 0.0168
ш, 0.0000, 0.0109
щ, 0.0000, 0.0047
ъ, 0.0003, 0.0003
ы, 0.0374, 0.0000
ь, 0.0432, 0.0009
э, 0.0065, 0.0000
ю, 0.0025, 0.0079
я, 0.0330, 0.0126
ё, 0.0001, 0.0000


In [13]:
N=3
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
hmm = HMM()
hmm.fit(obs,pi,A,B)

for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}".format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 , 0.0978, 0.0422, 0.4187
а, 0.0000, 0.1968, 0.0000
б, 0.0327, 0.0000, 0.0104
в, 0.0716, 0.0000, 0.0467
г, 0.0372, 0.0000, 0.0143
д, 0.0453, 0.0009, 0.0218
е, 0.0000, 0.1535, 0.0484
ж, 0.0188, 0.0000, 0.0050
з, 0.0272, 0.0000, 0.0186
и, 0.0000, 0.1427, 0.0095
й, 0.0000, 0.0000, 0.0335
к, 0.0581, 0.0000, 0.0318
л, 0.0921, 0.0000, 0.0323
м, 0.0433, 0.0000, 0.0249
н, 0.1308, 0.0000, 0.0351
о, 0.0000, 0.2674, 0.0026
п, 0.0464, 0.0000, 0.0220
р, 0.0866, 0.0000, 0.0126
с, 0.0434, 0.0002, 0.0958
т, 0.1042, 0.0000, 0.0289
у, 0.0000, 0.0646, 0.0018
ф, 0.0021, 0.0000, 0.0016
х, 0.0122, 0.0000, 0.0077
ц, 0.0076, 0.0000, 0.0004
ч, 0.0202, 0.0000, 0.0149
ш, 0.0147, 0.0000, 0.0038
щ, 0.0076, 0.0000, 0.0000
ъ, 0.000

Once again, the matrix is more sparse. б, в, г, з ,й, ж, л, are some characters appearing to be vowels.